<a href="https://colab.research.google.com/github/damianiRiccardo90/BHP/blob/master/C11-Offensive_Forensics/Vulnerability_Reconnaissance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# *__Vulnerability Reconnaissance__*

Now let's use Volatility to discover whether the target VM has vulnerabilities we may be able to exploit. The __malfind__ plug-in checks for process memory ranges that potentially contain injected code. _Potential_ is the key word here, the plug-in is looking for regions of memory that have permissions to read, write, and execute. It is worthwhile to investigate these processes since they may enable us to leverage some malware that is already available. Alternatively, we may be able to overwrite those regions with our own malware.

```
PS> vol -f WinDev2007Eval-7d959ee5.vmem windows.malfind
Volatility 3 Framework 1.2.0-beta.1
Progress: 33.01 Scanning primary2 using PdbSignatureScanner
PID Process Start VPN End VPN Tag Protection CommitCharge
1336 timeserv.exe 0x660000 0x660fff VadS PAGE_EXECUTE_READWRITE 1
2160 MsMpEng.exe 0x16301690000 0x1630179cfff VadS PAGE_EXECUTE_READWRITE 269
2160 MsMpEng.exe 0x16303090000 0x1630318ffff VadS PAGE_EXECUTE_READWRITE 256
2160 MsMpEng.exe 0x16304a00000 0x16304bfffff VadS PAGE_EXECUTE_READWRITE 512
6484 FreeDesktopClo 0x2320000 0x2320fff VadS PAGE_EXECUTE_READWRITE 1
5340 Microsoft.Powe 0x2c2502c0000 0x2c2502cffff VadS PAGE_EXECUTE_READWRITE 15
```

We've encountered a couple of potential problems. The __timeserv.exe__ process (__PID 1336__) is part of the freeware known as __FreeDesktopClock__ (__PID 6484__). These processes are not necessarily a problem as long as they're installed under _C:\Program Files_. Otherwise, the process may be malware masquerading as a clock.

Using a search engine, you will find that the process __MsMpEng.exe__ (__PID 2160__) is an anti-malware service. Even though these processes contain writable and executable memory regions, they don't appear to be dangerous. Perhaps we could make these processes dangerous by writing shellcode into those memory regions, so it's worth taking note of them.

The __netscan__ plug-in provides a list of all the network connections the machine had at the time of the snapshot, as shown next. Anything that looks suspicious we may be able to leverage in an attack.

```
PS>vol -f WinDev2007Eval-7d959ee5.vmem windows.netscan
Volatility 3 Framework 1.2.0-beta.1
Progress: 33.01 Scanning primary2 using PdbSignatureScanner
Offset Proto LocalAddr LocalPort ForeignAdd ForeignPort State PID Owner
0xa50bb7a13d90 TCPv4 0.0.0.0 4444 0.0.0.0 0 LISTENING 7124 nc64.exe #[1]
0xa50bb9f4c310 TCPv4 0.0.0.0 7680 0.0.0.0 0 LISTENING 1776 svchost.exe
0xa50bb9f615c0 TCPv4 0.0.0.0 49664 0.0.0.0 0 LISTENING 564 lsass.exe
0xa50bb9f62190 TCPv4 0.0.0.0 49665 0.0.0.0 0 LISTENING 492 wininit.exe
0xa50bbaa80b20 TCPv4 192.168.28.128 50948 23.40.62.19 80 CLOSED #[2]
w0xa50bbabd2010 TCPv4 192.168.28.128 50954 23.193.33.57 443 CLOSED
0xa50bbad8d010 TCPv4 192.168.28.128 50953 99.84.222.93 443 CLOSED
0xa50bbaef3010 TCPv4 192.168.28.128 50959 23.193.33.57 443 CLOSED
0xa50bbaff7010 TCPv4 192.168.28.128 50950 52.179.224.121 443 CLOSED
0xa50bbbd240a0 TCPv4 192.168.28.128 139 0.0.0.0 0 LISTENING
```

We see some connections from the local machine (192.168.28.128), apparently to a couple of web servers __[2]__; these connections are now closed. More important are the connections marked __LISTENING__. The ones that are ownbed by recognizable Windows processes (__svchost__, __lsass__, __wininit__) may be okay, but the __nc64.exe__ process is unknown __[1]__. It is listening on port 4444, and it's well worth taking a deeper look by using our __netcat__ substitute from Chapter 2 to probe that port.